### Quora Question Pairs 

Can you identify question pairs that have the same intent?


    Import libraries

In [35]:
import pandas as pd
import numpy as np
import nltk

    Load the dataset

In [36]:
training_labels = pd.read_csv('train_labels.csv')
test_data = pd.read_csv('test_data.csv')

# print(training_labels)
# print(test_data)

    Preprocessing

In [50]:
def text_regularize(dataframe):
    print ('***** Regularizing *****')
    
 
def text_filtering(dataframe):
    print ('***** Filtering *****')
    
    # import regular expression
    import re
    
    # remove stopwords
    
    # remove special symbols
        


print('Split test data')
test_q1 = test_data['question1']
test_q2 = test_data['question2']       
    
print('========= Clean testing data question 1 ====================')
# Do cleaning
text_regularize(test_q1)
text_filtering(test_q1)

print('========= Clean testing data question 1 ====================')
# Do cleaning
text_regularize(test_q2)
text_filtering(test_q2)

# Save as new file
test_q1.to_csv('test_q1.csv')

print('========= Finish preprocessing of data ==========')

Split test data
========= Clean testing data question 1 ====================
***** Regularizing *****
***** Filtering *****
========= Clean testing data question 1 ====================
***** Regularizing *****
***** Filtering *****
========= Finish preprocessing of data ==========


    Word2vec